#### Bank Loan Performance  Review

##### Project Object
This project is to demonstrate the use of SQL and Pandas for data analysis

##### Project requirements
##### Overview KPIs
    Sliced by All, loan status, term, Purpose, average Debt-to-Income Ratio (DTI), home ownership
1.	Total loan application
2.	Total loan amount
3.	Average loan amount


##### Good and Bad Loans KPIs
	Good loans(Loans with status 'Fully Paid' and 'Current.')
    Bad loans(Loans with status 'Charged Off')
1.	Count
2.	total amount and percentage of good loans
3.	average years of credit history
4.	average Debt-to-Income Ratio (DTI)
5.	average credit score
6.	average number of open accounts
7.	average number of credit problems
8.	average maximum open credit.


In [1]:
import pandas as pd
from sqlalchemy import create_engine
pd.set_option("display.max_columns", None)

In [2]:
# Establish a database connection
with open('project_secret.txt', 'r') as file:
    driver = file.readline().strip()
    server_name = file.readline().strip()
    database = file.readline().strip()
    username = file.readline().strip()
    password = file.readline().strip()
    Table_credit_train = file.readline().strip()
    Table_credit_test = file.readline().strip()

# To create a SQLAlchemy engine
engine = create_engine(f'mssql+pyodbc://{username}:{password}@{server_name}/{database}?driver={driver}')


In [3]:
# To check if there are duplicated rows based on loan_id
query = f"""
with duplicate AS (
	Select Loan_ID, count(*) AS duplicateLoanIDCounter from {Table_credit_train}
	group by Loan_ID
	having count(*) > 1
)

select d.duplicateLoanIDCounter, t.* from {Table_credit_train} AS t
JOIN
duplicate d on d.loan_id = t.loan_id
"""
df_duplicated = pd.read_sql(query, engine)

In [4]:
print(df_duplicated['duplicateLoanIDCounter'].describe())

count    36002.0
mean         2.0
std          0.0
min          2.0
25%          2.0
50%          2.0
75%          2.0
max          2.0
Name: duplicateLoanIDCounter, dtype: float64


In [5]:
# To reload the data without the duplicate rows
query = f"""
WITH numbered_rows AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY Loan_ID ORDER BY Loan_ID) AS row_num
    FROM {Table_credit_train}
)
SELECT *
FROM numbered_rows
WHERE row_num = 1;
"""
df_non_duplicated = pd.read_sql(query, engine)

In [6]:
print(f"Number of non-duplicated clean data is {len(df_non_duplicated)}")

Number of non-duplicated clean data is 81999


In [7]:
df_final=df_non_duplicated.copy()

In [8]:
df_final.columns = [x.lower() for x in df_final.columns] # Change field name to lower case
df_final.head()

,loan_id,customer_id,loan_status,current_loan_amount,term,credit_score,annual_income,years_in_current_job,home_ownership,purpose,monthly_debt,years_of_credit_history,months_since_last_delinquent,number_of_open_accounts,number_of_credit_problems,current_credit_balance,maximum_open_credit,bankruptcies,tax_liens,row_num
0,0000757f-a121-41ed-b17b-162e76647c1f,dde79588-12f0-4811-bab0-e2b07f633fcd,Fully Paid,258082,Short Term,746.0,950475.0,4 years,Rent,Debt Consolidation,6748.419922,11.500000,NaN,12,0,330429,815782.0,0.0,0.0,1
1,0000afa6-8902-4f8f-b870-25a8fdad0aeb,e49c1a82-a0f7-45e8-9f46-2f75c43f9fbc,Charged Off,541486,Long Term,NaN,NaN,6 years,Rent,Business Loan,10303.509766,17.600000,73.0,7,0,268337,372988.0,0.0,0.0,1
2,00020fb0-6b8a-4b3a-8c72-9c4c847e8cb6,c9decd06-16f7-44c3-b007-8776f2a9233d,Fully Paid,99999999,Short Term,742.0,1230440.0,3 years,Home Mortgage,Debt Consolidation,11073.959961,26.799999,NaN,11,0,168720,499642.0,0.0,0.0,1
3,00045ecd-59e9-4752-ba0d-679ff71692b3,b7bce684-b4b0-4b29-af66-eae316bce573,Fully Paid,260986,Short Term,734.0,1314838.0,10+ years,Own Home,Debt Consolidation,16325.940430,30.299999,NaN,7,0,189221,373890.0,0.0,0.0,1
4,0004f37b-5859-40f6-98d0-367aa3b3f3f1,f662b062-5fa5-463d-b5c0-4e36d09fcab1,Fully Paid,301818,Short Term,NaN,NaN,1 year,Own Home,Home Improvements,14770.219727,13.600000,2.0,12,0,127680,1173370.0,0.0,0.0,1


In [9]:
len(df_final)

81999

In [10]:
df_final.describe()

,current_loan_amount,credit_score,annual_income,monthly_debt,years_of_credit_history,months_since_last_delinquent,number_of_open_accounts,number_of_credit_problems,current_credit_balance,maximum_open_credit,bankruptcies,tax_liens,row_num
count,8.199900e+04,64939.000000,6.493900e+04,81999.000000,81999.000000,37378.000000,81999.000000,81999.000000,8.199900e+04,8.199700e+04,81824.000000,81991.000000,81999.0
mean,1.210213e+07,1168.636366,1.376561e+06,18330.633109,18.296783,35.064236,11.114489,0.161441,2.936203e+05,7.935358e+05,0.113463,0.028064,1.0
std,3.219825e+07,1633.006359,1.119818e+06,12127.700799,7.043774,22.021222,4.981266,0.473148,3.726145e+05,9.208747e+06,0.344674,0.254642,0.0
min,1.080200e+04,585.000000,7.662700e+04,0.000000,3.600000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,1.0
25%,1.792340e+05,711.000000,8.478180e+05,10117.595215,13.500000,16.000000,8.000000,0.000000,1.133160e+05,2.804560e+05,0.000000,0.000000,1.0
50%,3.079120e+05,732.000000,1.170590e+06,16075.330078,17.000000,32.000000,10.000000,0.000000,2.099310e+05,4.777740e+05,0.000000,0.000000,1.0
75%,5.193320e+05,743.000000,1.649248e+06,23811.370117,21.799999,51.000000,14.000000,0.000000,3.669945e+05,7.984900e+05,0.000000,0.000000,1.0
max,1.000000e+08,7510.000000,1.655574e+08,435843.281250,70.500000,176.000000,76.000000,15.000000,3.287897e+07,1.539738e+09,7.000000,15.000000,1.0


In [11]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81999 entries, 0 to 81998
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   loan_id                       81999 non-null  object 
 1   customer_id                   81999 non-null  object 
 2   loan_status                   81999 non-null  object 
 3   current_loan_amount           81999 non-null  int64  
 4   term                          81999 non-null  object 
 5   credit_score                  64939 non-null  float64
 6   annual_income                 64939 non-null  float64
 7   years_in_current_job          81999 non-null  object 
 8   home_ownership                81999 non-null  object 
 9   purpose                       81999 non-null  object 
 10  monthly_debt                  81999 non-null  float64
 11  years_of_credit_history       81999 non-null  float64
 12  months_since_last_delinquent  37378 non-null  float64
 13  n

#

### KPIs

#### Overview KPIs
    Sliced by All, loan status, term, Purpose, average Debt-to-Income Ratio (DTI), home ownership
1.	Total loan application
2.	Total loan amount
3.	Average loan amount

#### Sliced by All

In [13]:
def overviewQuery(field):
    if field == '':
        fieldName = ''
        groupBy = ''
    else:
        fieldName = field + ','
        groupBy = 'GROUP BY ' + field
        
    q = f"""
    WITH numbered_rows AS (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY Loan_ID ORDER BY Loan_ID) AS row_num
        FROM {Table_credit_train}
    ),
    clean_data AS (
        SELECT *
        FROM numbered_rows
        WHERE row_num = 1
    )
    SELECT
        {fieldName}
        count(*) AS total_loan_application,
        avg(monthly_debt/(annual_income/12)) AS average_dti,
        sum(current_loan_amount) AS total_loan_amount,
        avg(current_loan_amount) AS average_loan_amount
    FROM clean_data
    WHERE monthly_debt IS NOT NULL AND annual_income IS NOT NULL
    {groupBy}
    ORDER BY total_loan_amount DESC
    ;
    """
    return q

In [14]:
query = overviewQuery('')
result = pd.read_sql(query, engine)
print(result)

   total_loan_application  average_dti  total_loan_amount  average_loan_amount
0                   64943     0.171335       985224281833             15170600


.

#### Sliced by Loan term

In [15]:
query = overviewQuery('term')
result = pd.read_sql(query, engine)
print(result)

         term  total_loan_application  average_dti  total_loan_amount  \
0  Short Term                   48169     0.167716       817636839833   
1   Long Term                   16775     0.181709       167288863511   

   average_loan_amount  
0             16974337  
1              9972510  


#### Sliced by Loan Purpose

In [16]:
query = overviewQuery('purpose')
result = pd.read_sql(query, engine)
print(result)

                 purpose  total_loan_application  average_dti  \
0     Debt Consolidation                   51273     0.178168   
1                  Other                    5822     0.151520   
2      Home Improvements                    3815     0.140516   
3              Buy a Car                     854     0.135396   
4          Medical Bills                     704     0.163635   
5          Business Loan                     978     0.133506   
6              Buy House                     418     0.130663   
7            Take a Trip                     353     0.162591   
8         major_purchase                     241     0.133262   
9         small_business                     187     0.131794   
10               wedding                      70     0.135555   
11                moving                      96     0.150552   
12  Educational Expenses                      70     0.125600   
13              vacation                      56     0.173008   
14      renewable_energy 

#### Sliced by Loan home ownership

In [17]:
query = overviewQuery('home_ownership')
result = pd.read_sql(query, engine)
print(result)

  home_ownership  total_loan_application  average_dti  total_loan_amount  \
0  Home Mortgage                   31585     0.170558       497862373778   
1           Rent                   27257     0.170998       396747871696   
2       Own Home                    5962     0.177102        88000656953   
3   HaveMortgage                     141     0.163825         2314906473   

   average_loan_amount  
0             15762620  
1             14555815  
2             14760257  
3             16417776  


.

### Good and Bad Loans KPIs

In [18]:
def goodLoadQuery(field):
    if field == '':
        fieldName = ''
        groupBy = ''
    else:
        fieldName = field + ','
        groupBy = 'GROUP BY ' + field
        
    q = f"""
    WITH numbered_rows AS (
        SELECT
            *,
            ROW_NUMBER() OVER (PARTITION BY Loan_ID ORDER BY Loan_ID) AS row_num
        FROM {Table_credit_train}
    ),
    clean_data AS (
        SELECT *
        FROM numbered_rows
        WHERE row_num = 1
    )
    SELECT
        CASE
            WHEN loan_status <> 'Charged Off' THEN 'Good loan'
            ELSE 'Bad loan'
        END AS loan_classification,
        count(*) AS total_loan_application,
        avg(monthly_debt/(annual_income/12)) AS average_dti,
        sum(current_loan_amount) AS total_loan_amount,
        avg(current_loan_amount) AS average_loan_amount,
        avg(years_of_credit_history) AS average_years_of_credit_history,
        avg(credit_score) AS average_credit_score,
        avg(number_of_open_accounts) AS average_number_of_open_accounts
    FROM clean_data
    WHERE monthly_debt IS NOT NULL AND annual_income IS NOT NULL
    {groupBy}
    ORDER BY total_loan_amount DESC
    ;
    """
    return q

#### Sliced by Loan Status

In [19]:
query = goodLoadQuery('loan_status')
result = pd.read_sql(query, engine)
print(result)

  loan_classification  total_loan_application  average_dti  total_loan_amount  \
0           Good loan                   47729     0.165371       979722282594   
1            Bad loan                   17210     0.187832         5600275208   

   average_loan_amount  average_years_of_credit_history  average_credit_score  \
0             20526771                        18.435781                   723   
1               325408                        17.891540                  2402   

   average_number_of_open_accounts  
0                               11  
1                               11  
